In [83]:
#!pip install yake
#!pip install pdfplumber
#!pip install unstructured
# !pip install pi-heif
# !pip install "unstructured[local-inference,pdf]"
# !pip install langchain-mongodb



In [84]:
import os
os.environ["USER_AGENT"] = "MyLangchainApp/1.0 (saahmathworks@gmail.com)"
from langchain.document_loaders import (
    PyPDFLoader,
    Docx2txtLoader,
    CSVLoader,
    WebBaseLoader,
    DirectoryLoader,
    TextLoader
)
from langchain_community.document_loaders import UnstructuredPDFLoader
from langchain_community.document_loaders import PDFPlumberLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain
from langchain_community.embeddings import HuggingFaceEmbeddings
# from langchain_community.vectorstores import MongoDBAtlasVectorSearch
from langchain_mongodb.vectorstores import MongoDBAtlasVectorSearch
from pymongo import MongoClient
import yake
import datetime
import pymongo
import pytesseract
from pdf2image import convert_from_path
from langchain.schema import Document 
import yake


In [50]:
# Load environment variables
import os
from dotenv import load_dotenv
load_dotenv(dotenv_path='./laws/.env')

# correctly access the variable by its name, which is 'mongo_uri'
MONGO_URI = os.environ.get('mongo_uri')
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
# Ensure API keys and URI are set
if not OPENAI_API_KEY:
    raise ValueError("OPENAI_API_KEY not found. Please set it in your .env file.")
if not MONGO_URI:
    raise ValueError("MONGO_URI not found. Please set it in your .env file.")

print("Configuration loaded successfully.")

Configuration loaded successfully.


In [51]:
DB_NAME = "legal_db"
COLLECTION_NAME = "legal_documents"

In [65]:
# --- Initialize OpenAI Embeddings ---
# This will be used to generate vectors from your document chunks
embedding_model = OpenAIEmbeddings(model="text-embedding-ada-002", openai_api_key=OPENAI_API_KEY)
print("OpenAI Embeddings model initialized.")

# --- Connect to MongoDB Atlas ---
try:
    client = MongoClient(MONGO_URI)
    db = client[DB_NAME]
    collection = db[COLLECTION_NAME]
    print(f"Connected to MongoDB Atlas database '{DB_NAME}', collection '{COLLECTION_NAME}'.")
    # Ping the database to ensure connection is active
    client.admin.command('ping')
    print("MongoDB connection test successful.")
except Exception as e:
    print(f"Error connecting to MongoDB Atlas: {e}")
    # You might want to halt execution if connection fails
    raise

OpenAI Embeddings model initialized.
Connected to MongoDB Atlas database 'legal_db', collection 'legal_documents'.
MongoDB connection test successful.


In [ ]:
# ---------------------------
# 1. Load document with OCR fallback
# ---------------------------

def load_document(file_path, metadata, lang="fra"):
    """
    Load a PDF document. Try PDFPlumber first, and if it fails or returns empty text,
    fall back to OCR using Tesseract.
    Returns a list of LangChain Document objects.
    """
    try:
        loader = PDFPlumberLoader(file_path)
        documents = loader.load()

        # Check if text content is non-empty
        if not any(doc.page_content.strip() for doc in documents):
            raise ValueError("Empty text, switching to OCR...")

        print(f"✅ PDFPlumber loaded {len(documents)} pages successfully.")
        return documents

    except Exception as e:
        print(f"⚠️ PDFPlumber failed ({e}), switching to OCR...")

        # OCR fallback
        try:
            images = convert_from_path(file_path)
            ocr_docs = []
            for i, img in enumerate(images):
                text = pytesseract.image_to_string(img, lang=lang).strip()

                if not text:
                    print(f"⚠️ OCR page {i+1}/{len(images)} is empty, skipping.")
                    continue

                ocr_docs.append(Document(
                    page_content=text,
                    metadata={**metadata, "page": i + 1}
                ))

                print(f"📄 OCR page {i+1}/{len(images)} extracted ({len(text)} chars).")

            print(f"✅ OCR extracted {len(ocr_docs)} pages with text.")
            return ocr_docs

        except Exception as ocr_error:
            print(f"❌ OCR failed: {ocr_error}")
            return []



In [ ]:
documents_to_process = [
    {
        "file_path": "./loi-2002-07.pdf",
        "metadata": {
            "titre": "Loi N° 2002-07 portant Code des personnes et de la famille",
            "pays": "Bénin",
            "categorie": "Code des personnes et de la famille",
            "langue": "Français",
            "date_publication": datetime.datetime(2004, 8, 24),
            "url_source": "https://sgg.gouv.bj/doc/loi-2002-07/"
        }
    }
]

In [ ]:
docs = []
for doc_info in documents_to_process:
    file_path = doc_info["file_path"]
    metadata = doc_info["metadata"]

    ocr_docs = load_document(file_path, metadata, lang="fra")
    docs.extend(ocr_docs) 


In [ ]:
print("data type of docs is; ", type(docs), "and it length is: ", len(docs))

In [ ]:
docs[6]

In [53]:
# ---------------------------
# 2. Split text into chunks
# ---------------------------
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.schema import Document

def split_documents(documents, chunk_size=1000, chunk_overlap=150):
    """
    Split a list of LangChain Document objects into smaller chunks.
    
    Args:
        documents (list[Document]): The documents to split (from load_document).
        chunk_size (int): Maximum size of each text chunk in characters.
        chunk_overlap (int): Number of characters to overlap between chunks.

    Returns:
        list[Document]: A new list of Document objects, where each page is split
                        into smaller chunks with preserved metadata.
    """
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
        length_function=len
    )

    chunks = text_splitter.split_documents(documents)
    print(f"✅ Split {len(documents)} documents into {len(chunks)} chunks.")
    return chunks


In [54]:
# Apply the splitter
chunks = split_documents(docs, chunk_size=1000, chunk_overlap=150)




✅ Split 124 documents into 494 chunks.


In [55]:
# Quick check: print first 300 chars of the first 2 chunks
for i, chunk in enumerate(chunks[15:20]):
    print(f"\n--- Chunk {i+1} (page {chunk.metadata.get('page', 'N/A')}) ---")
    print(f"\n--- Chunk {i+1} (page {chunk.metadata}) ---")
    print(chunk.page_content[:700], "...")


--- Chunk 1 (page 5) ---

--- Chunk 1 (page {'titre': 'Loi N° 2002-07 portant Code des personnes et de la famille', 'pays': 'Bénin', 'categorie': 'Code des personnes et de la famille', 'langue': 'Français', 'date_publication': datetime.datetime(2004, 8, 24, 0, 0), 'url_source': 'https://sgg.gouv.bj/doc/loi-2002-07/', 'page': 5}) ---
Article 28 : L'affaire est instruite et jugée en chambre du conseil. Tous les actes de la
procédure ainsi que les expéditions et extraits desdits actes sont dispensés du timbre et enregistrés
gratis. Si le tribunal estime que le décès n'est pas suffisamment établi, il peut ordonner toute mesure
d'information complémentaire et requérir notamment une enquête administrative sur les circonstances
de la disparition. Si le décès est déclaré, sa date doit être fixée en tenant compte des présomptions
tirées des circonstances de la cause et, à défaut, au jour de la disparition. Cette date ne doit jamais être
indéterminée.

Article 29 : Le dispositif du jugement déc

In [56]:
print("data type of chunks is; ", type(chunks), "and it length is: ", len(chunks))

data type of chunks is;  <class 'list'> and it length is:  494


In [57]:
import yake
from langchain.docstore.document import Document

# ---------------------------
# 3. Generate embeddings (LangChain version)
# ---------------------------
def embed_chunks(chunks, embedding_model, lang="fr"):
    """
    Generate embeddings and extract keywords for each document chunk using
    LangChain's OpenAIEmbeddings and YAKE for keyword extraction.

    Args:
        chunks (list[Document]): List of LangChain Document objects (already split).
        embedding_model (OpenAIEmbeddings): LangChain OpenAIEmbeddings instance.
        lang (str): Language for keyword extraction (default = 'fr').

    Returns:
        list[dict]: List of dictionaries, each containing:
                    - "contenu": raw chunk text
                    - "vecteur_embedding": OpenAI embedding vector
                    - "mots_cles": extracted keywords
                    - "metadata": metadata from the original document
    """
    kw_extractor = yake.KeywordExtractor(lan=lang, n=3, top=5)
    embedded_chunks = []

    for chunk in chunks:
        content = chunk.page_content.strip()
        if not content:
            continue

        keywords = [kw for kw, _ in kw_extractor.extract_keywords(content)]

        try:
            # ✅ LangChain embeddings
            embedding_vector = embedding_model.embed_query(content)
        except Exception as e:
            print(f"❌ Error creating embedding: {e}. Skipping chunk.")
            continue

        embedded_chunks.append({
            "contenu": content,
            "vecteur_embedding": embedding_vector,
            "mots_cles": keywords,
            "metadata": chunk.metadata
        })

    print(f"✅ Generated embeddings for {len(embedded_chunks)} chunks.")
    return embedded_chunks


In [58]:
# Step 3: Embed chunks
embedded_chunks = embed_chunks(chunks, embedding_model)

✅ Generated embeddings for 494 chunks.


In [59]:
type(embedded_chunks)

list

In [60]:
type(embedded_chunks[5])

dict

In [97]:
#embedded_chunks[5]

In [66]:

# ---------------------------
# 4. Save to MongoDB
# ---------------------------
def save_to_mongodb(embedded_chunks, collection, base_metadata, batch_size=100):
    """
    Save embedded document chunks into MongoDB in batches.
    Ensures compliance with schema validation for 'legal_documents'.

    Args:
        embedded_chunks (list[dict]): List of chunks with embeddings and metadata.
        collection (pymongo.collection.Collection): MongoDB collection object.
        base_metadata (dict): Base metadata shared by all chunks
                              (must include titre, pays, categorie, langue, date_publication, url_source).
        batch_size (int): Number of documents per batch insert.

    Returns:
        None
    """
    if not embedded_chunks:
        print("⚠️ No chunks to save.")
        return

    required_fields = ["titre", "pays", "categorie", "langue", "date_publication", "url_source"]

    # --- Validate base_metadata against schema ---
    for field in required_fields:
        if field not in base_metadata:
            raise ValueError(f"❌ Missing required metadata field: {field}")

    documents_to_insert = []
    # Check for existing documents based on the unique url_source
    existing_document = collection.find_one({"url_source": base_metadata['url_source']})
    
    if existing_document:
        print(f"📄 Document for '{base_metadata['titre']}' already exists. Skipping insertion.")
        return
        
    for item in embedded_chunks:
        doc_to_save = {
            **base_metadata,                            # global metadata
            "contenu": item["contenu"],                 # must be string
            "vecteur_embedding": item["vecteur_embedding"], # must be array
            "mots_cles": item["mots_cles"]              # must be array of strings
        }

        # The 'metadata' from the chunk is not included to align with your schema.
        # This prevents a validation error.

        # Ensure 'contenu' is string (Mongo validator requires it)
        if not isinstance(doc_to_save["contenu"], str):
            doc_to_save["contenu"] = str(doc_to_save["contenu"])

        documents_to_insert.append(doc_to_save)

    try:
        # --- Insert in batches ---
        for i in range(0, len(documents_to_insert), batch_size):
            batch = documents_to_insert[i:i + batch_size]
            collection.insert_many(batch)
            print(f"✅ Inserted batch {i//batch_size + 1} "
                  f"({len(batch)} docs) for '{base_metadata['titre']}'.")
    except pymongo.errors.BulkWriteError as bwe:
        print(f"❌ Bulk write error: {bwe.details}")
    except pymongo.errors.OperationFailure as e:
        print(f"❌ Failed to insert into MongoDB: {e}")

In [67]:
# Step 4: Save to MongoDB
save_to_mongodb(embedded_chunks, collection, metadata)

📄 Document for 'Loi N° 2002-07 portant Code des personnes et de la famille' already exists. Skipping insertion.


In [39]:
# Step 4: Save to MongoDB
save_to_mongodb(embedded_chunks, collection, metadata)

✅ Inserted batch 1 (100 docs) for 'Loi N° 2002-07 portant Code des personnes et de la famille'.
✅ Inserted batch 2 (100 docs) for 'Loi N° 2002-07 portant Code des personnes et de la famille'.
✅ Inserted batch 3 (100 docs) for 'Loi N° 2002-07 portant Code des personnes et de la famille'.
✅ Inserted batch 4 (100 docs) for 'Loi N° 2002-07 portant Code des personnes et de la famille'.
✅ Inserted batch 5 (94 docs) for 'Loi N° 2002-07 portant Code des personnes et de la famille'.


In [103]:
from langchain.embeddings.openai import OpenAIEmbeddings

embedding_model = OpenAIEmbeddings(
    model="text-embedding-ada-002",
    openai_api_key=OPENAI_API_KEY
)

vectorstore = MongoDBAtlasVectorSearch(
    collection=collection,
    embedding=embedding_model,
    index_name="vector_index",
    text_key="contenu",
    embedding_key="vecteur_embedding"
)

retriever = vectorstore.as_retriever(search_kwargs={"k": 10})


In [104]:
doc = collection.find_one()
print("Content:", doc.get("contenu"))
print("Vector length:", len(doc.get("vecteur_embedding", [])))


Content: FE.-
REPUBLIQUE DU BENIN

LOI N° 2002-07 DU 24 AOUT 2004

portant Code des personnes et de la famille.

-  L’ASSEMBLEE NATIONALE à délibéré et adopté en sa séance du 07 juin 2002, puis
en sa séance du 14 juin 2004, suite à la décision DCC 02-144 du 23 décembre 2002 de la
Cour Constitutionnelle Pour mise en conformité à la Constitution ;

- Suite à la Décision de conformité à la Constitution DCC 04-083 du 20 août 2004 de ja
Cour Constitutionnelle ;

- LE PRESIDENT DE LA REPUBLIQUE promulgue la loi dont la teneur suit :

LIVRE PREMIER : DES PERSONNES

TITRE PREMIER : DES PERSONNES PHYSIQUES ET MORALES

CHAPITRE 1 : DES DISPOSITIONS GENERALES

Article 1”  : Toute personne humaine, sans distinction aucune notamment de race, de couleur,
de sexe, de réligion, de langue, d’opinion politique ou de toute autre opinion , d’origine nationale
où sociale, de fortune, de naissance ou de toute autre situation, est sujet de droit, de sa naissance
à son décès.
Vector length: 1536


In [105]:
query = "La décision qui autorise le changement de nom profite au requérant et à ses enfants"

docs = retriever.get_relevant_documents(
    query
)


# --- 3. Inspect results ---
for i, doc in enumerate(docs, start=1):
    print(f"\n📄 Result {i}")
    print("Titre:", doc.metadata.get("titre"))
    print("Pays:", doc.metadata.get("pays"))
    print("Catégorie:", doc.metadata.get("categorie"))
    print("Contenu (preview):", doc.page_content[:200], "…")



📄 Result 1
Titre: Loi N° 2002-07 portant Code des personnes et de la famille
Pays: Bénin
Catégorie: Code des personnes et de la famille
Contenu (preview): L'adjonction ou la radiation de prénoms peut être autorisée dans les mêmes conditions.

@ La requête est présentée au tribunal dans le ressort duquel le requérant est né, et au
tribunal de première in …

📄 Result 2
Titre: Loi N° 2002-07 portant Code des personnes et de la famille
Pays: Bénin
Catégorie: Code des personnes et de la famille
Contenu (preview): expédition du jugement ou de l'arrêt, accompagnée d'un certificat délivré par le greffier et duquel il
résulte que le jugement ou l'arrêt est devenu définitif.

Article 11 : Il peut être procédé à des …

📄 Result 3
Titre: Loi N° 2002-07 portant Code des personnes et de la famille
Pays: Bénin
Catégorie: Code des personnes et de la famille
Contenu (preview): , Toutefois, un surnom ou un pseudonyme peut être choisi pour préciser l'identité d'une
personne, mais il ne fait päs partie d

In [107]:

# --- 5. Query with pre_filter ---
query = "actes de naissance et registres civils"

# pre_filter is a dict for exact matching on metadata fields
pre_filter = {
    "pays": "Bénin"
    # "titre": "some title"  # optional
}

docs = retriever.get_relevant_documents(query, pre_filter=pre_filter)

# --- 6. Inspect results ---
for i, doc in enumerate(docs, start=1):
    print(f"\n📄 Result {i}")
    print("Titre:", doc.metadata.get("titre"))
    print("Pays:", doc.metadata.get("pays"))
    print("Catégorie:", doc.metadata.get("categorie"))
    print("Contenu (preview):", doc.page_content[:200], "…")


📄 Result 1
Titre: Loi N° 2002-07 portant Code des personnes et de la famille
Pays: Bénin
Catégorie: Code des personnes et de la famille
Contenu (preview): Celle-ci se fait sur les registres de l'état civil du lieu :
- de naissance, pour les actes de reconnaissance ;
- du dernier domicile du père ou, si le père est inconnu, de la mère, pour les actes de  …

📄 Result 2
Titre: Loi N° 2002-07 portant Code des personnes et de la famille
Pays: Bénin
Catégorie: Code des personnes et de la famille
Contenu (preview): La présentation de ce registre peut être exigée à tout moment par l'officier de l'état civil du
lieu où est situé l'établissement, ainsi que par les autorités administratives qui y sont expressément
a …

📄 Result 3
Titre: Loi N° 2002-07 portant Code des personnes et de la famille
Pays: Bénin
Catégorie: Code des personnes et de la famille
Contenu (preview): Sur chaque feuille portant la même lettre que la première du nom de l'intéressé, seront
inscrits, au moment de la rédaction de